In [ ]:
from sctoolbox.utils.jupyter import bgcolor, _compare_version

# change the background of input cells
bgcolor("PowderBlue", select=[2, 6, 8, 10, 12, 14])

nb_name = "GSEA.ipynb"

_compare_version(nb_name)

# Gene Set Enrichment Analysis (GSEA)
<hr style="border:2px solid black"> </hr>

## 1 - Description

**Requires an anndata object with precomputed marker genes. Marker genes can be generated with the marker gene notebook (`general_notebooks/group_markers.ipynb`).**

**Move this notebook into the notebook folder (e.g. `rna_analysis/notebooks/`) of the respective analysis before using it!**

The main function of this notebook is to perform enrichment analysis described as "\[...\] a computational method for inferring knowledge about an input gene set by comparing it to annotated gene sets representing prior biological knowledge."\[[source](https://maayanlab.cloud/Enrichr/help#background)\]. So in other words, the goal is to collect enriched GO pathways for clusters of cells (e.g. cell types) based on cluster defining sets of marker genes.  
Available methods in this notebook are [Enrichr](https://maayanlab.cloud/Enrichr/) and [GSEA preank](https://www.genepattern.org/modules/docs/GSEAPreranked/1#gsc.tab=0), which are both implemented in [GSEApy](https://github.com/zqfang/GSEApy).

"enrichr performs a hypergeometric test comparing the set of "significant" genes against the "universe" (or background) genes. GSEA is a Komolgorov-Smirnov test on the whole ranked gene list, testing if some category (e.g., a KEGG pathway) is more abundant at the top of the list than expected by chance." - [source](https://www.biostars.org/p/9468150/)

---

## 2 - Setup

In [ ]:
import sctoolbox.utils as utils
import sctoolbox.tools as tools
import sctoolbox.plotting as pl
from sctoolbox import settings

import pandas as pd
import gseapy as gp
import tqdm
import matplotlib.pyplot as plt
import warnings

settings.settings_from_config("config.yaml", key="GSEA")

---

## 3 - General Input

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# Set input anndata
last_notebook_adata = "anndata_marker.h5ad"

# Define the dataset species!
organism = "human"

---

## 4 - Load anndata

In [ ]:
adata = utils.adata.load_h5ad(last_notebook_adata)

with pd.option_context("display.max.rows", 5, "display.max.columns", None):
    display(adata)
    display(adata.obs)
    display(adata.var)

---

## 5 - Select library

**Molecular Function**  
Molecular-level activities performed by gene products. Molecular function terms describe activities that occur at the molecular level, such as “catalysis” or “transport”. GO molecular function terms represent activities rather than the entities (molecules or complexes) that perform the actions, and do not specify where, when, or in what context the action takes place. Molecular functions generally correspond to activities that can be performed by individual gene products (i.e. a protein or RNA), but some activities are performed by molecular complexes composed of multiple gene products. Examples of broad functional terms are catalytic activity and transporter activity; examples of narrower functional terms are adenylate cyclase activity or Toll-like receptor binding. To avoid confusion between gene product names and their molecular functions, GO molecular functions are often appended with the word “activity” (a protein kinase would have the GO molecular function protein kinase activity).  

**Cellular Component**  
A location, relative to cellular compartments and structures, occupied by a macromolecular machine. There are two ways in which the gene ontology describes locations of gene products: (1) the cellular anatomical entities, in which a gene product carries out a molecular function. Cellular anatomical entities includes cellular structures such as the plasma membrane and the cytoskeleton, as well as membrane-enclosed cellular compartments such as the mitochondrion, and (2) the stable macromolecular complexes of which they are parts, e.g., the clathrin complex.  

**Biological Process**  
The larger processes, or ‘biological programs’ accomplished by multiple molecular activities. Examples of broad biological process terms are DNA repair or signal transduction. Examples of more specific terms are pyrimidine nucleobase biosynthetic process or glucose transmembrane transport. Note that a biological process is not equivalent to a pathway. At present, the GO does not try to represent the dynamics or dependencies that would be required to fully describe a pathway.

-- https://geneontology.org/docs/ontology-documentation/

**List of available librarys**

In [ ]:
[db for db in gp.get_library_name(organism) if db.startswith("GO")]

**List of available `marker_keys`**

In [ ]:
[k for k in adata.uns.keys() if k.startswith("rank_")]

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# key for marker table in adata.uns
clustering = "leiden_0.1"  # The marker_key is usually formatted as rank_genes_<clustering>.
marker_key = f"rank_feature_{clustering}"  # The marker_key under which the ranked gene list is stored in adata.uns

# Select method. Available options: 'prerank', 'enrichr'
method = "prerank"

# Choose public library to use
library_name = "GO_Biological_Process_2023"

# If custom gene sets and background should be used set here.
# The public library will be ignored if custom_gene_set is given.
custom_gene_set = None  # {"Pathway 1": ["Gene1", "Gene2",...], ...}

# enrichr specific parameters
# To use a custom background for the public gene set library only set custom_background.
custom_background = None  # ["Gene 1", "Gene 2", ....]

# prerank specific parameters
threads = 4  # Number of threadsused by prerank function
min_size = 5  # Minimum allowed number of genes from gene set also the data set.
max_size = 1000  # Maximum allowed number of genes from gene set also the data set.
permutation_num = 1000  # Number of permutations.
seed = 0  # Seed for prerank run

overwrite = False

---

## 6 - Run gene set enrichment analysis

The results will be stored in the anndata object (`adata.uns['sctoolbox']['gsea']['enrichment_table']`).

In [ ]:
tools.gsea.gene_set_enrichment(adata,
                               method=method,
                               marker_key=marker_key,
                               organism=organism,
                               gene_sets=custom_gene_set,
                               background=custom_background,
                               library_name=library_name,
                               seed=seed,
                               min_size=min_size,
                               max_size=max_size,
                               permutation_num=permutation_num,
                               threads=threads,
                               inplace=True,
                               overwrite=overwrite)

---

## 7 - Plotting
<hr style="border:2px solid black"> </hr>

### 7.1 - Dotplot

The dotplot shows the top enriched pathways per cluster. The size of the dot indicates the fraction of genes in the cluster that match the pathway and the color of the dot indicates statistical significance (higher is better).

#### 7.1.1 Combined plot

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# Dotplot
figsize = (15, 20)  # Set figure size for dotplot
top_term = 10  # Number of top pathways
sizes = (50, 200)  # Min, max size for dots
x_label_rotation = 0  # Set rotation angle for x-tick labels
cutoff = 0.05  # Terms with `column` value < cut-off are shown.
title = f"Top {top_term} upregulated pathways per cluster"
title_size = 16

---

In [ ]:
ax = pl.gsea.gsea_dot(adata,
                      cutoff=cutoff,
                      top_term=top_term,
                      figsize=figsize,
                      sizes=sizes,
                      title=title,
                      title_size=title_size,
                      x_label_rotation=x_label_rotation,
                      x="Cluster",
                      save=f"GSEA_top-{top_term}_per_cluster.pdf",
                      report=f"01_GSEA_top-{top_term}_per_cluster.png")

#### 7.1.2 Dotplot per cluster

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# Dotplot
figsize_clust_dot = (10, 10)  # Set figure size for dotplot
top_up = 5  # Number of upregulated pathways to be plotted
top_down = 5  # Number of downregulated pathways to be plotted
size_clust_dot = (50, 200)  # Min, max size for dots
cutoff_clust_dot = 0.05  # Terms with `column` value < cut-off are shown.

---

In [ ]:
pl.gsea.cluster_dotplot(adata,
                        figsize=figsize_clust_dot,
                        top_up=top_up,
                        top_down=top_down,
                        sizes=size_clust_dot,
                        cutoff=cutoff_clust_dot)

---

### 7.2 - Term dotplot
The term dotplot focuses on a single term/pathway and thus shows individual genes instead of pathways on the y-axis. A Z-Score is applied to the mean gene expression per cluster to highlight differences in expression between the clusters (x-axis). A pathway of interest can be selected using the plot above.

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# Term dotplot
term = "Actin Filament Organization (GO:0007015)"  # The GO term of interest.
groups = None  # Only show selected groups on x-axis e.g. ["a", "b"]. None to show all.
figsize_term_dot = None # Set figure size for term dotplot e.g. (10, 5).
save_term_dot = f"term_dotplot_{term}_{clustering}.pdf"

# Data layer (advanced)
# decide which processing state of the data should be used for computation
# available layers can be seen above, e.g. "norm" to use normalized data
# it is recommended to use raw or normalized data for statistical testing
layer = "norm"

---

In [ ]:
if term:
    pl.gsea.term_dotplot(term=term,
                         adata=adata,
                         groupby=clustering,
                         gene_col="Lead_genes" if method == "prerank" else "Genes",
                         groups=groups,
                         figsize=figsize_term_dot,
                         save=save_term_dot,
                         layer=layer,
                         report='02_' + save_term_dot.replace('pdf', 'png'))

### 7.3 Network plot
The network plot shows connections between enriched pathways per cluster. In the plot the node size corresponds to the percentage of gene overlap in a certain term of interest. The colour of the node corresponds to the significance of the enriched terms and the edge size corresponds to the number of genes that overlap between two connected nodes.

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# Significance cutoff for (sig_column), e.g. "FDR q-val"
# All terms with a sig_column value > cutoff are filtered.
cutoff = 0.05 

scale = 1  # Scale factor for node positions
figsize_network = None # Set figure size for term dotplot e.g. (10, 5). Set to None to use default.
ncols = 3  # Number of columns for network plots
save_network = f"pathway_network.pdf"

---

In [ ]:
pl.gsea.gsea_network(adata,
                     cutoff=cutoff,
                     scale=scale,
                     figsize=figsize_network,
                     ncols=ncols,
                     save=save_network,
                     report='03_' + save_network.replace('pdf', 'png'))

## 8 - Save adata

In [ ]:
utils.adata.save_h5ad(adata, "anndata_gsea.h5ad")